# An Introduction to pyvista-xarray

**pyvista-xarray** provides a bridge between [xarray](https://docs.xarray.dev/) and [PyVista](https://docs.pyvista.org/), enabling interactive 3D visualization of xarray data structures. By importing `pvxarray`, a `.pyvista` accessor is registered on `xarray.DataArray` objects, giving you direct access to 3D mesh creation and plotting.

This notebook covers:
1. Creating a **RectilinearGrid** from 1D coordinates (most common use case)
2. Auto-detection of spatial coordinates using CF conventions
3. Working with raster data via rioxarray

> **Tip:** For curvilinear grids (2D/3D coordinates) see the radar and ocean model examples. For scattered point data, use `mesh_type="points"`.

In [ ]:
import numpy as np
import xarray as xr

import pvxarray  # noqa: F401 - registers the .pyvista accessor

## Quick Start: Air Temperature

The xarray tutorial dataset `air_temperature` has coordinates named `lat` and `lon`. Since pyvista-xarray understands CF conventions and common coordinate names, you can call `.pyvista.mesh()` without specifying any axes - they are auto-detected:

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
da = ds.air.isel(time=0)  # Select a single timestep

# Auto-detect coordinates and plot in 3D
da.pyvista.plot(cpos="xy")

### Inspecting Detected Coordinates

You can see which coordinates were detected using the `.pyvista.axes` property:

In [ ]:
# Show detected CF axis mapping
print("Detected axes:", da.pyvista.axes)
print("Spatial coords:", da.pyvista.spatial_coords)

### Working with the Mesh Object

Instead of plotting directly, you can grab the mesh for use with PyVista's full API:

In [ ]:
mesh = da.pyvista.mesh(x="lon", y="lat")
mesh

## 3D RectilinearGrid

When your data has three spatial dimensions with 1D coordinates (the most common case for gridded data), pyvista-xarray creates a `RectilinearGrid`. This is the most memory-efficient mesh type since PyVista/VTK can share the underlying arrays with xarray without copying data.

In [ ]:
lon = np.array([-99.83, -99.32])
lat = np.array([42.25, 42.21])
z = np.array([0, 10])
temp = 15 + 8 * np.random.randn(2, 2, 2)

ds = xr.Dataset(
    {"temperature": (["z", "x", "y"], temp)},
    coords={
        "lon": (["x"], lon),
        "lat": (["y"], lat),
        "z": (["z"], z),
    },
)

mesh = ds.temperature.pyvista.mesh(x="lon", y="lat", z="z")
mesh.plot()

## Raster Data with rioxarray

[rioxarray](https://corteva.github.io/rioxarray/) extends xarray with rasterio-based geospatial functionality. Since rioxarray DataArrays have standard `x` and `y` coordinates, pyvista-xarray works seamlessly with them.

Here we load a GeoTIFF, reproject it, and create a 3D surface by warping by the scalar values:

In [ ]:
import rioxarray

# From https://download.osgeo.org/geotiff/samples/spot/chicago/
da = rioxarray.open_rasterio("data/UTM2GTIF.tiff")
da = da.rio.reproject("EPSG:3857")

# Select a single band and create a mesh
mesh = da.isel(band=0).pyvista.mesh(x="x", y="y")

# Warp by scalar values to create a 3D surface
mesh.warp_by_scalar().plot(jupyter_backend="server")